# MONAI Deploy App SDKとMONAI Bundleでデプロイアプリを作成する。

このチュートリアルでは、MONAIで学習し、[MONAI Bundle](https://docs.monai.io/en/latest/bundle_intro.html) フォーマットでパッケージされたPyTorchモデル用の臓器分割アプリケーションを作成する方法を紹介します。

AIモデルの展開には、たとえ研究用であっても臨床画像ネットワークとの統合が必要です。つまり、AIを導入するアプリケーションは、標準ベースの画像プロトコル、特に放射線画像についてはDICOMプロトコルをサポートする必要があります。

通常、DICOMネットワーク通信は、DICOM TCP/IPネットワークプロトコルまたはDICOMWebのいずれかで、DICOMデバイスまたはサービス（例：MONAI Deploy Informatics Gateway）によって処理されるので、展開アプリケーション自体は、入力としてDICOM Part10ファイルを使用して、AI結果をDICOM Part10ファイル（複数）に保存するだけで良いのです。セグメンテーションのユースケースでは、AI結果のDICOMインスタンスファイルはDICOM SegmentationオブジェクトまたはDICOM RT Structure Setであり、分類ではDICOM Structure ReportやDICOM Encapsulated PDFになります。

画像ネットワークと統合され、モダリティやPACS（Picture Archiving and Communications System）からDICOMインスタンスを受け取る場合、AI導入アプリケーションは、複数のシリーズを含むDICOM検査全体を処理しなければならず、その画像の間隔は学習済みモデルによって予想されたものとは異なる可能性があります。MONAI DeployアプリケーションSDKは、このようなケースに一貫して効率的に対応するため、DICOM検査を解析し、アプリケーション定義のルールに従って特定のシリーズを選択し、選択したDICOMシリーズをドメイン固有の画像フォーマットに変換し、適切なDICOM属性を表すメタデータを付加するオペレータと呼ばれるクラスを提供します。その後、画像は前処理段階において、ピクセルデータをテンソルとして推論に使用する前に、間隔、方向、強度などを正規化するためにさらに処理される。

以下のセクションでは、MONAI Deploy アプリケーション SDK を使って MONAI Deploy アプリケーションパッケージを作成する方法と、MONAI Bundle に組み込まれた推論オペレータを使って MONAI Bundle の脾臓 CT Segmentation PyTorch モデルで推論を行う方法を説明します。


:::{note}
DICOM Part 10 ファイルがない場合、3D Slicer などのオープンソースソフトウェアを使用して、NIfTI ファイルを DICOM シリーズに変換することができます。

この例を簡単に実行するために、DICOMファイルと[MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo)で公開されている[Spleen CT Segmentation MONAI Bundle](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation) をパッケージ化してGoogle Driveで共有しています。

:::

## ApplicationクラスでのOperatorの作成とその接続

5つのOperatorで構成されるアプリケーションを実装します。

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: フォルダパス ([`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html)) **Output(dicom_stiles)**: フォルダパス (`DataPath`) を入力する。
    - **Output(dicom_study_list)**: メモリ上の DICOM study のリスト (List[[`DICOMStudy`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DICOMStudy.html)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: メモリ上の DICOM スタディのリスト (List[[`DICOMStudy`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DICOMStudy.html)])
    - **Input(selection_rules)**: 選択ルール(Dict)
    - **Output(study_selected_series_list)**: メモリ上の DICOM series オブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**: メモリ上の DICOM シリーズオブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
    - **Output(image)**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
- **MonaiBundleInferenceOperator**:
    - **Input(image)**: メモリ上のイメージオブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Output(pred)**: メモリー上のイメージオブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
- **DICOMSegmentationWriterOperator**:
    - **Input(seg_image)**: メモリ上のセグメンテーションイメージオブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Input(study_selected_series_list)**: メモリー上の DICOM シリーズオブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
    - **Output(dicom_seg_instance)**: ファイルパス（[`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html))


:::{note}
DICOMSegmentationWriterOperator` は、患者の属性と DICOM Study レベルの属性を使用するために、セグメンテーション画像と元の DICOM シリーズのメタデータの両方を必要とします。
:::

アプリケーションのワークフローは以下の通りです。

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB

    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : study_selected_series_list...study_selected_series_list
    DICOMSeriesToVolumeOperator --|> MonaiBundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    MonaiBundleInferenceOperator --|> DICOMSegmentationWriterOperator : pred...seg_image


    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        study_selected_series_list(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>study_selected_series_list : IN_MEMORY
        image(out) IN_MEMORY
    }
    class MonaiBundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
```

### 環境のセットアップ


In [2]:
# MONAIなど、アプリケーションに必要な画像処理パッケージのインストール
!python -c "import monai" || pip install --upgrade -q "monai"
!python -c "import torch" || pip install -q "torch>=1.10.2"
!python -c "import numpy" || pip install -q "numpy>=1.21"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"
!python -c "import typeguard" || pip install -q "typeguard>=2.12.1"

# MONAI Deploy App SDK パッケージのインストール
!python -c "import monai.deploy" || pip install --upgrade -q "monai-deploy-app-sdk"

Note: 更新されたパッケージを使用するために、Jupyterカーネルを再起動する必要がある場合があります。

### Google Driveから入力ファイル、モデル・バンドルファイルをダウンロード・抽出

In [3]:
# テストデータおよびMONAIバンドルzipファイルのダウンロード
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ"

# Webブラウザまたはgdownでai_spleen_bundle_dataのzipファイルをダウンロードした後、ai_spleen_bundle_dataのzipファイルをダウンロードしてください。
!unzip -o "ai_spleen_seg_bundle_data.zip"

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14821 sha256=7f8a0488ff5936dfdce82319eb68703e25a19f31fb50a1d6d81d732254ae54b6
  Stored in directory: c:\users\ryo\appdata\local\pip\cache\wheels\ed\8a\de\15273923b1f5adacc09306cc48dcedc6b082892569afb2dfc7
Successfully built gdown


You should consider upgrading via the 'D:\repos\study\monaijp\.venv\Scripts\python.exe -m pip install --upgrade pip' command.
Downloading...
From: https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ
To: d:\repos\study\monaijp\monai-deploy-app-sdk\notebooks\ai_spleen_seg_bundle_data.zip

  0%|          | 0.00/79.4M [00:00<?, ?B/s]
  1%|          | 524k/79.4M [00:00<00:24, 3.24MB/s]
  5%|▍         | 3.67M/79.4M [00:00<00:04, 15.9MB/s]
  7%|▋         | 5.77M/79.4M [00:00<00:04, 15.9MB/s]
 12%|█▏        | 9.44M/79.4M [00:00<00:04, 17.1MB/s]
 15%|█▍        | 11.5M/79.4M [00:00<00:05, 13.4MB/s]
 17%|█▋        | 13.6M/79.4M [00:01<00:05, 11.5MB/s]
 24%|██▍       | 18.9M/79.4M [00:01<00:03, 19.1MB/s]
 27%|██▋       | 21.5M/79.4M [00:01<00:03, 16.0MB/s]
 33%|███▎      | 26.2M/79.4M [00:01<00:04, 13.0MB/s]
 36%|███▌      | 28.3M/79.4M [00:02<00:04, 11.9MB/s]
 43%|████▎     | 34.1M/79.4M [00:02<00:02, 18.4MB/s]
 47%|████▋     | 37.2M/79.4M [00:02<00:02, 15.8MB/s]
 54%|█████▎    | 42.

### インポートの設定

ApplicationとOperatorを定義するために必要なクラスやデコレータをインポートしましょう。

In [4]:
import logging

# SegmentDescription属性の設定に必要です。App SDK パッケージに含まれないため、直接インポートする。
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import IOMapping, MonaiBundleInferenceOperator


d:\repos\study\monaijp\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### モデルバンドル推論オペレータの入力と出力の決定

MONAI Bundleは、基本的にTorchScriptで記述されたPyTorchモデルに、モデルネットワークと処理仕様を記述する追加のメタデータを加えたものです。この演算子は、MONAIユーティリティを使用してMONAI Bundleを解析し、入出力処理と推論に必要なオブジェクトを自動的にインスタンス化するため、MONAIトランスフォーム、インファラー、そしてそれらの依存関係に依存します。

各 Operator クラスは、ベースとなる [Operator](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html) クラスを継承し、その入出力プロパティは [@input](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.input.html)/[@output](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.output.html) デコレーターで指定することができます。`MonaiBundleInferenceOperator` クラスでは、入力と出力をモデルネットワークと同じ名前とデータ型で定義する必要があります。現在のリリースでは、`IOMapping`オブジェクトを使用して、オペレータの入出力をモデルネットワークの入出力と同じ名前に接続しています。今後のリリースでは、アプリSDKの制限を解除し、自動化する予定です。

Spleen CT Segmentation モデルネットワークは、"image"という名前の入力と、"pred"という名前の出力を持ち、どちらも画像タイプで、すべて App SDK [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) にマッピングすることが可能である。この情報の一部は、通常、この[example](https://github.com/Project-MONAI/model-zoo/blob/dev/models/spleen_ct_segmentation/configs/metadata.json)に見られるように、バンドル内のモデルメタデータのnetwork_data_format属性を調べることによって取得されます。

### アプリケーションクラスの作成

アプリケーションクラスは以下のようになります。

[Application](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html) クラスを継承した `App` クラスが定義されています。]

アプリの要件（リソースやパッケージの依存関係）は、[@resource](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.resource.html) と [@env](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.env.html) というデコレータで指定できます。

ベースクラスのメソッドである `compose` はオーバーライドされます。DICOM 解析、シリーズ選択（現在のリリースで最初のシリーズを選択）、ピクセルデータからボリューム画像への変換、セグメンテーションインスタンスの生成に必要なオブジェクトと、モデル固有の `SpleenSegOperator` が生成される。実行パイプラインは、これらのオブジェクトを <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a> で連結した Directed Acyclic Graph として作成される。

In [5]:
@resource(cpu=1, gpu=1, memory="7Gi")
# pip_packages には、requirements.txt ファイルへのパス(str)、またはパッケージのリスト を指定します。
# monai pkg はこのクラスでは必要ありません。代わりに、インクルードされている演算子で必要です。
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        """アプリケーションのインスタンスを作成します。"""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # このメソッドは、実行するベースクラスを呼び出す。単に呼び出すだけであれば、省略可能です。
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """アプリ固有の演算子を作成し、処理DAG内で連鎖させる。"""

        logging.info(f"Begin {self.compose.__name__}")

        # SDK 組み込みの演算子と同様に、カスタム演算子を作成します。
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # MONAI Bundleをサポートし、推論を自動化する推論オペレータを作成する。
        # IOMappingのラベルは、事前処理と事後処理で入力キーと予測キーに一致する。
        # model_name は、アプリが1つのモデルしか持っていない場合、オプションで指定します。
        # アプリがMAPにパッケージされるとき、オペレータはinit時にバンドルの解析を完了し、
        # バンドルから解析されたオプションのパッケージ情報をパッケージャに提供し、
        # パッケージャがMAPのドッカーイメージにパッケージをインストールできるようにします。
        # 出力IOTypeをDISKに設定することは、リーフオペレータにのみ有効で、この例ではそうではありません。
        #
        # Pertinent MONAI Bundle:
        # https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation

        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
        )

        # 各セグメントに必要なセグメント記述を、実際のアルゴリズムと該当する臓器/組織で提供するDICOM Seg ライターを作成する。
        # segment_label, algorithm_name, algorithm_version は DICOM VR LO タイプで、64 文字以内である。
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.1.0",
            )
        ]
        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            segment_descriptions=segment_descriptions, custom_tags=custom_tags
        )
        # 処理パイプラインを作成し、ソースとデスティネーションの演算子を指定し、
        # 前者の出力が後者の入力と名前と型の両方で一致することを確認します。
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # dicom_seg_writerは2つの入力を必要とし、それぞれがソースオペレータから来ることに注意してください。
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {"pred": "seg_image"})
        # サーフェスメッシュSTL変換演算子を作成し、必要に応じて以下の2行をアンコメントしてアプリの実行フローに追加してください。
        # stl_conversion_op = STLConversionOperator(output_file="stl/spleen.stl")
        # self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {"pred": "image"})

        logging.info(f"End {self.compose.__name__}")


## ローカルでアプリを実行する

Jupyterノートブック上でアプリを実行します。CT AbdomenシリーズのDICOMファイルが `dcm` に、Torch Scriptのモデルが `model.ts` に存在する必要があることに注意してください。あなたの環境での実際のパスを使用してください。


In [6]:
app = AISpleenSegApp()

app.run(input="dcm", output="output", model="model.ts")

2022-10-26 00:08:52,936 - Begin compose
2022-10-26 00:08:52,941 - End compose
2022-10-26 00:08:52,943 - Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 6848, Operator ID: 60dcac76-37eb-4ac3-aed7-97394228affa)


ValueError: Required input folder does not exist.

Once the application is verified inside Jupyter notebook, we can write the above Python code into Python files in an application folder.

The application folder structure would look like below:

```bash
my_app
├── __main__.py
└── app.py
```

In [6]:
# Create an application folder
!mkdir -p my_app

### app.py

In [7]:
%%writefile my_app/app.py

# Copyright 2021-2022 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import IOMapping, MonaiBundleInferenceOperator

# from monai.deploy.operators.stl_conversion_operator import STLConversionOperator  # import as needed.


@resource(cpu=1, gpu=1, memory="7Gi")
# pip_packages can be a string that is a path(str) to requirements.txt file or a list of packages.
# The monai pkg is not required by this class, instead by the included operators.
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        """アプリケーションのインスタンスを作成します。"""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # このメソッドは、実行するベースクラスを呼び出す。単に呼び出すだけであれば、省略可能です。
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """アプリ固有の演算子を作成し、処理DAG内で連鎖させる。"""

        logging.info(f"Begin {self.compose.__name__}")

        # SDK 組み込みの演算子と同様に、カスタム演算子を作成します。
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name is optional when the app has only one model.
        # The bundle_path argument optionally can be set to an accessible bundle file path in the dev
        # environment, so when the app is packaged into a MAP, the operator can complete the bundle parsing
        # during init to provide the optional packages info, parsed from the bundle, to the packager
        # for it to install the packages in the MAP docker image.
        # Setting output IOType to DISK only works only for leaf operators, not the case in this example.
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation
        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.1.0",
            )
        ]
        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            segment_descriptions=segment_descriptions, custom_tags=custom_tags
        )

        # Create the processing pipeline, by specifying the source and destination operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # dicom_seg_writerは2つの入力を必要とし、それぞれがソースオペレータから来ることに注意してください。
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {"pred": "seg_image"})
        # Create the surface mesh STL conversion operator and add it to the app execution flow, if needed, by
        # uncommenting the following couple lines.
        # stl_conversion_op = STLConversionOperator(output_file="stl/spleen.stl")
        # self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {"pred": "image"})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

if __name__ == "__main__":
    # Creates the app and test it standalone. When running is this mode, please note the following:
    #     -m <model file>, for model file path
    #     -i <DICOM folder>, for input DICOM CT series folder
    #     -o <output folder>, for the output folder, default $PWD/output
    # e.g.
    #     monai-deploy exec app.py -i input -m model/model.ts
    #
    logging.basicConfig(level=logging.DEBUG)
    app_instance = AISpleenSegApp(do_run=True)


Overwriting my_app/app.py


```python
if __name__ == "__main__":
    AISpleenSegApp(do_run=True)
```

The above lines are needed to execute the application code by using `python` interpreter.

### \_\_main\_\_.py

\_\_main\_\_.py is needed for <a href="../../developing_with_sdk/packaging_app.html#required-arguments">MONAI Application Packager</a> to detect the main application code (`app.py`) when the application is executed with the application folder path (e.g., `python simple_imaging_app`).

In [8]:
%%writefile my_app/__main__.py
from app import AISpleenSegApp

if __name__ == "__main__":
    AISpleenSegApp(do_run=True)

Overwriting my_app/__main__.py


In [9]:
!ls my_app

app.py	__main__.py  __pycache__  spleen_seg_operator.py


In this time, let's execute the app in the command line.

In [10]:
!python my_app -i dcm -o output -m model.ts

2022-10-18 18:05:59,753 - Begin compose
2022-10-18 18:05:59,754 - End compose
2022-10-18 18:05:59,754 - Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1020777, Operator ID: 2771ad3e-33fb-4038-b88a-4d853d049a81)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 1020777, Operator ID: 44d22322-ff83-4a88-be70-5cc4223694e3)
[2022-10-18 18:06:00,668] [INFO] (root) - Finding series for Selection named: CT Series
[2022-10-18 18:06:00,668] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-10-18 18:06:00,668] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-10-18 18:06:00,668] [INFO] (root) - On attribute: 'StudyDescription' to match 

Above command is same with the following command line:

In [11]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
!monai-deploy exec my_app -i dcm -o output -m model.ts

2022-10-18 18:06:21,346 - Begin compose
2022-10-18 18:06:21,349 - End compose
2022-10-18 18:06:21,349 - Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1020839, Operator ID: cd6de9f0-8a20-4c8e-ac40-e5bc1fa7c7e2)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 1020839, Operator ID: 99f613a7-c853-4076-bc0e-e12db550cb78)
[2022-10-18 18:06:22,381] [INFO] (root) - Finding series for Selection named: CT Series
[2022-10-18 18:06:22,381] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-10-18 18:06:22,382] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-10-18 18:06:22,382] [INFO] (root) - On attribute: 'StudyDescription' to match 

In [12]:
!ls output

1.2.826.0.1.3680043.10.511.3.12041615526490041922994303524987331.dcm
1.2.826.0.1.3680043.10.511.3.46164851841087364253778499084246156.dcm
1.2.826.0.1.3680043.10.511.3.59253586144990251681167260565313540.dcm


## Packaging app

Let's package the app with [MONAI Application Packager](/developing_with_sdk/packaging_app).

In [13]:
!monai-deploy package -b nvcr.io/nvidia/pytorch:21.11-py3 my_app --tag my_app:latest -m model.ts

[2022-10-18 18:06:43,671] [INFO] (root) - Begin compose
[2022-10-18 18:06:43,673] [INFO] (root) - End compose
Building MONAI Application Package... Done
[2022-10-18 18:07:05,945] [INFO] (app_packager) - Successfully built my_app:latest


:::{note}
Building a MONAI Application Package (Docker image) can take time. Use `-l DEBUG` option if you want to see the progress.
:::

We can see that the Docker image is created.

In [14]:
!docker image ls | grep my_app

my_app                                                                latest                                     cb63d46f6b85   4 seconds ago       15.1GB


## Executing packaged app locally

The packaged app can be run locally through [MONAI Application Runner](/developing_with_sdk/executing_packaged_app_locally).

In [15]:
# Copy DICOM files are in 'dcm' folder

# Launch the app
!monai-deploy run my_app:latest dcm output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "my_app:latest" is available...

Checking for MAP "my_app:latest" locally
"my_app:latest" found.

Reading MONAI App Package manifest...
--> Verifying if "nvidia-docker" is installed...

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2022-10-19 01:07:20,302 - Begin compose
2022-10-19 01:07:20,304 - End compose
2022-10-19 01:07:20,304 - Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1, Operator ID: 2b5b3f81-8010-442f-b119-85c5ed42a6a6)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSer

In [16]:
!ls output

1.2.826.0.1.3680043.10.511.3.10344779550124226469948949035444422.dcm
1.2.826.0.1.3680043.10.511.3.12041615526490041922994303524987331.dcm
1.2.826.0.1.3680043.10.511.3.46164851841087364253778499084246156.dcm
1.2.826.0.1.3680043.10.511.3.59253586144990251681167260565313540.dcm
